In [1]:
import pandas as pd
import numpy as np

In [2]:
with open("day9.txt", "r") as f:
    lines = f.readlines()
    lines = [list(l.rstrip()) for l in lines]
    cave = np.array(lines).astype(int)

In [3]:
### functions
def neighbours(ix,boundaries):
    """
    gets the index for the up, down, left, right
    """ 
    row,col = ix[0], ix[1]
    up = (row-1, col)
    down = (row+1, col)
    left = (row, col-1)
    right = (row, col+1)
    nbours = [up, down, left, right]
    
    x,y = boundaries
    out = [-1, x,y]
    safe = []
    for n in nbours:
        if checkbounds(n, out):
            safe.append(n)
    
    return safe

def checkbounds(ix, notsafe):
    """ 
    check if the neighouring spot is out of bounds
    """ 
    safe = True
    for i in ix:
        if i == notsafe[0]:
            safe=False
    if ix[0] == notsafe[1]:
        safe=False
    if ix[1] == notsafe[2]:
        safe=False
            
    return safe

def check_low(ix, cavemap):
    """
    if it is lower than the up, down, left, right neighbours
    """    
    lowest=True
    current = cavemap[ix]
    checks = neighbours(ix, cavemap.shape)
    
    for c in checks:
        compare = cavemap[c]
        if compare<current:
            lowest=False
    
    return lowest

def trickle(ix, cavemap):
    """
    finds the lowest of the neighbours and drop/flow towards it
    """
    current = cavemap[ix]
    checks = neighbours(ix, cavemap.shape)
    drop=1e9
    for c in checks:
        compare = cavemap[c]
        if compare<current and compare<drop:
            drop=compare
            newdrop=c
    return newdrop

In [4]:
### part 1 
low = set()
### loop through every coordinate on the cave map and trickle down from there. 
for row in range(cave.shape[0]):
    for col in range(cave.shape[1]):
        ix = (row, col)
        movement =0
        while not check_low(ix, cave):
            movement+=1
            ix = (trickle(ix, cave))
        ## must check for flow movement
        if movement>0:
            low.add(ix)
        
## sum up
total = 0
for l in low:
    total+=cave[l]+1
    #print(cave[l], l)
    
print(total)

480


In [5]:
def climb(ix, cavemap):
    """
    finds the higher neighbours and climb up it
    """
    current = cavemap[ix] ## the cave height
    checks = neighbours(ix, cavemap.shape) ## neighbours
    scale=[]
    for c in checks:
        compare = cavemap[c] ## neighbour height
        if compare>current and compare!=9:
            scale.append(c)
            
    return scale

def findnewspot(spotlist, cavemap):
    """
    loops through list to find new spots
    """
    newspot = []
    for s in spotlist:
        newspot.extend(climb(s, cavemap))
    return newspot

def basin(ix, cavemap):
    """
    map out basin.
    """
    mapped = set()
    mapped.add(ix)
    adj = neighbours(ix, cavemap.shape)
    adj = [a for a in adj if cavemap[a]!=9]
    ## map nearby 
    [mapped.add(a) for a in adj]
    ## survey
    nspots = findnewspot(adj, cavemap)

    finishmap = False
    while not finishmap: 
        if (set(nspots).difference(mapped)):
            [mapped.add(a) for a in nspots]
            nspots = findnewspot(nspots, cavemap)
        else:
            finishmap=True
           
    return mapped



In [6]:
basinsize = [] 
for l in low:
    base = basin(l, cave)
    basinsize.append(len(base))

basinsize.sort(reverse=True)
basinsize[0] * basinsize[1]* basinsize[2]

1045660